In this tutorial, we'll teach you about the basics of working with finite fields. Finite fields are very useful mathematical objects that pop up everywhere in cryptography. What is a "finite field" you might ask? Before we get into this, it might be useful to first review the mathematical concept of a "field".

Don't be thrown off by the mathematical terminology. You've seen fields before. The most common field is the real numbers, often written as 

$$\mathbb{R}$$

Here are some common examples of real numbers

$$\pi, e, 1.244353$$

What can we do with real numbers? Well, we can add them, we can subtract them, multiply them, and divide them. Here are some examples of real number operations

$$\pi^2, \pi + \pi, \pi/\pi$$

A field is simply any set where you can add, sutract, multiply, and divide numbers. (You might protest you can't divide by 0. That's a special case handled differently). What are some other examples of fields you might have seen? Well likely the rational numbers

$$\mathbb{Q}$$

Members of this field are simply fractions. Note that you can add, subtract, multiply and divide fractions

$$\frac{a}{b} + \frac{c}{d} = \frac{ad + bc}{bd}$$
$$\frac{a}{b} \times \frac{c}{d} = \frac{ac}{bd}$$
$$\frac{1}{\frac{a}{b}} = \frac{b}{a}$$

Ok, you might now be feeling that fields are pretty common. What are some things that aren't fields? Well for starters, the set of whole numbers isn't a field. Why not? You can add, subtract, and multiply whole numbers, but you can't divide them. For example

$$ \frac{1}{2} \not\in \mathbb{Z}$$

where we've used the mathematical notation for the set of whole numbers

$$\mathbb{Z}$$

This is well and good you might say, but what does this have to do with cryptography? It turns out that the first field that's very useful in cryptography is the modular numbers

$$\mathbb{Z}_p$$

What are modular numbers? It might be easiest to describe them relative to clocks. In a clock, numbers don't increment forever. Rather they wrap around. This makes sense, since each day "resets" naturally. With a little more detail, note how 4 hours after 9 o'clock is 1 o'clock and not 13 o'clock.

![modular numbers](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a4/Clock_group.svg/320px-Clock_group.svg.png)

Modular numbers work the exact same way. In fact, if you want to model the numbers on a clock, the corresponding set of modular numbers is

$$\mathbb{Z}_{12}$$

How do addition and multiplication work? Well, simply put, add and multiply as usual and "wrap around" appropriately. Let's get some code going that demonstrates this. We're going to make use of the `starks` package to do some of this math for us.

In [3]:
import starks
from starks.modp import IntegersModP

The `IntegersModP` class allows us to construct modular numbers. Let's try making some!

In [4]:
mod12 = IntegersModP(12)
a = mod12(3)
print(a)

3


Ok, this seems unexciting so far. Let's try doing some basic arithmetic with `a`

In [5]:
print(a + a)
print(a + a + a)
print(a + a + a + a)

6
9
0


Ok now this is a little interesting. Adding `a` four times gets us back to zero! This shouldn't be surprising if we think about the clock face. 4 increments of 3 hours each is 12 hours. Going from 6pm to 6am takes us back to the same place on the clock. Ok, addition seems to make sense. Let's now take a look at multiplication. How does that work?

In [6]:
print(a*a)

9


Seems to make sense. Let's do one more multiplication and see what happens.

In [7]:
print(a*a*a)

3


Wait, how does this one work? Well, let's work out the math manually

$$ 3 * 3 * 3 = 27$$

Let's picture the clock face again. Say it's now 1 o'clock. What's the time 27 hours later? Well, 24 hours later, we're back at 1 o'clock, so 27 hours later it should be 4 o'clock. That is, the effective increment is 3 hours. In general, you can achieve this rounding effect by using the remainder operator `%`

In [8]:
(3 * 3 * 3) % 12

3

So all that happens in modular arithmetic is that we add, subtract and multiply as normal and then apply `%` to get back the remainder in the correct range. This isn't so bad right? Well then, what about division? This one turns out to be a little trickier. To understand why, let's briefly review what division means. Division is the "inverse" operation to multiplication. So if we have 

$$ a \times b = 1$$

Then we can say that 

$$ a = \frac{1}{b}$$

The issue with modular division is that there are sometimes numbers which don't have inverses this way. Let's return to 

$$\mathbb{Z}_{12}$$

What are the multiples of 2 here?

In [11]:
for i in range(12):
  print(mod12(2) * mod12(i))

0
2
4
6
8
10
0
2
4
6
8
10


Notice that 1 doesn't appear in this list! This means that there's no "multiplicative inverse" to the number 2 in 

$$\mathbb{Z}_{12}$$

The reason for that is that 2 and 12 share a common factor (that is, 2). Let's try the same with 7 instead since 7 and 12 share no common factors.

In [14]:
for i in range(12):
  print("Trying %d" % i)
  print(mod12(7)*mod12(i))

Trying 0
0
Trying 1
7
Trying 2
2
Trying 3
9
Trying 4
4
Trying 5
11
Trying 6
6
Trying 7
1
Trying 8
8
Trying 9
3
Trying 10
10
Trying 11
5


We're in luck! It turns out that 

$$7 * 7 = 49 \equiv 1\ (\textrm{mod} 12)$$

We've introduced a new bit of notation here. When we say

$$a \equiv b\ (\textrm{mod} 12)$$

we mean that both these numbers differ by a multiple of 12. For example

$$ 1 \equiv 13\ (\textrm{mod} 12)$$

Returning to our original point, that means 7 is its own inverse in 

$$\mathbb{Z}_{12}$$

Put another way

$$ \frac{1}{7} \equiv 7\ (\mathrm{mod} 12) $$

Now returning to the more general point, I've actually been lying to you a bit so far. In particular,

$$\mathbb{Z}_{12}$$

is not a field. This is becaususe division doesn't always work, like we saw for 2 above. Can we see when division would always work in modular arithmetic? We would need a number which doesn't share common factors with any of the numbers smaller than it. This is a prime number! More generally, let 
$$p$$
be a prime number. Then
$$\mathbb{Z}_p$$
the modular numbers mod $$p$$ will form a finite field. Let's play with this more with a simple example.

In [16]:
mod3 = IntegersModP(3)
for i in range(3):
  for j in range(3):
    print("%d * %d = %s" % (i, j, str(mod3(i) * mod3(j))))

0 * 0 = 0
0 * 1 = 0
0 * 2 = 0
1 * 0 = 0
1 * 1 = 1
1 * 2 = 2
2 * 0 = 0
2 * 1 = 2
2 * 2 = 1


Note here that 
$$1 * 1 \equiv 1\ (\textrm{mod} 3)$$
$$2 * 2 \equiv 1\ (\textrm{mod} 3)$$
(We don't count 0 since it never has a multiplicative inverse). This means that
$$\mathbb{Z}_3$$
is a field! This is the first example of a finite field that we've seen so far. Now that you've worked through this first tutorial, I recommend taking some time to play with other finite fields

## Exercises

1. Can you construct $$\mathbb{Z}_7$$ in Python?
2. What is $$\frac{1}{3}$$ in the finite field $$\mathbb{Z}_{19}$$?
3. Find a large prime number $$p$$ greater than $$2^{32}$$ Construct $$\mathbb{Z}_p$$ Try doing some simple arithmetic in this finite field. What do you notice about how the arithmetic works? When does "overflow" occur?